In [1]:
import numpy as np
import pandas as pd
import requests
! pip install BeautifulSoup4
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 1.1MB/s eta 0:00:01


## Creating dataframe for toronto neighborhoods

In [2]:
columns = ['PostalCode','Borough','Neighborhood']

toronto_neighborhood = pd.DataFrame(columns = columns)

toronto_neighborhood.head()

,PostalCode,Borough,Neighborhood


In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

In [5]:
for data in table_contents:
    postalCode = data['PostalCode']
    borough = data['Borough']
    neighborhood = data['Neighborhood']
    toronto_neighborhood = toronto_neighborhood.append({'Borough':borough,'Neighborhood':neighborhood,'PostalCode':postalCode},ignore_index=True)

toronto_neighborhood.shape

(103, 3)

### Ignore cells with a borough that is Not assigned 

In [6]:
toronto_neighborhood['Borough'].replace('', np.nan, inplace=True)

toronto_neighborhood.dropna(subset=['Borough'], inplace=True)



### Replacing not assigned neighborhoods with borough name

In [7]:
toronto_neighborhood['Neighborhood'].replace('', np.nan, inplace=True)

toronto_neighborhood['Neighborhood'].fillna(toronto_neighborhood['Borough'],inplace=True)

toronto_neighborhood.shape

(103, 3)

In [8]:
toronto_neighborhood

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Getting latitudes and longitudes in the dataframe

In [11]:
!pip install geocoder
import geocoder # import geocoder

     |████████████████████████████████| 102kB 16.3MB/s ta 0:00:01


In [ ]:
# initialize your variable to None
lat_lng_coords = None

#loop over all neighborhoods
for data in toronto_neighborhood['PostalCode']:

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(data))
      lat_lng_coords = g.latlng
    
    toronto_neighborhood['Latitude'] = lat_lng_coords[0]
    toronto_neighborhood['Longitude'] = lat_lng_coords[1]

toronto_neighborhood

#### using the csv file as the geocoder was taking a lot of time

In [8]:

toronto_lat_lng = pd.read_csv ('Geospatial_Coordinates.csv')
print(toronto_lat_lng)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [9]:
toronto_neighborhood_final=toronto_neighborhood.merge(toronto_lat_lng, left_on = 'PostalCode', right_on='Postal Code')

toronto_neighborhood_final

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


### Filtering dataframe with 'Toronto' in the Borough Name

In [12]:
result_df= toronto_neighborhood_final[toronto_neighborhood_final['Borough'].str.contains('Toronto')]

result_df.shape

(39, 6)

In [15]:
result_df.reset_index()

,index,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
2,15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
4,20,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
5,24,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
6,25,M6G,Downtown Toronto,Christie,M6G,43.669542,-79.422564
7,30,M5H,Downtown Toronto,"Richmond, Adelaide, King",M5H,43.650571,-79.384568
8,31,M6H,West Toronto,"Dufferin, Dovercourt Village",M6H,43.669005,-79.442259
9,35,M4J,East YorkEast Toronto,The Danforth East,M4J,43.685347,-79.338106


### Exploring neighborhoods using the foursquare api

#### Defining function to explore neighborhoods

In [21]:
CLIENT_ID = 'WG5MY2GPDYTQZQBQ1FFVGCAZC0CBPAPOXOXEKGQYWT54HPOH'
CLIENT_SECRET = 'I0H5XROS30VPBUVUGPJBU0DEDUDERJKP0ROW3REOHKXED5KY'
ACCESS_TOKEN = 'Y1DYEPFGWCWPMEOX4YI5N3ATPOE3C3S1LKHOX2FCOLAKSHOB'
VERSION='20180605'
LIMIT=100

In [24]:
def getNearbyVenues(names,latitudes,longitudes,radius = 500):
    venues_list = []
    for name,lat,lng in zip(names,latitudes,longitudes):
        # Create API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #return relevant info
        
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name'])for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude','Venue',
                             'Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

#### Getting nearby venues for the neighborhoods

In [35]:
toronto_venues = getNearbyVenues(names = result_df['Neighborhood'],latitudes = result_df['Latitude'],longitudes = result_df['Longitude'])

print(toronto_venues.shape)

toronto_venues.head()

(1583, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Count venues in each Neighborhood

In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,11,11,11,11,11,11


#### Number of unique categories in each neighborhood

In [29]:
print ('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 unique categories


#### One hot encoding based on venue categories for neighborhoods

In [40]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix = "",prefix_sep = "")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

Neigh = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', Neigh)

toronto_onehot.head()


,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### group rows by neighborhood and take mean of frequency of occurence of each category

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape
toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.016129
4,Christie,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


#### Lets put the top venues in a df

In [70]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st','nd','rd']

#create columns according to number of top venues

columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try: 
        columns.append('{}{}Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
    
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]= return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
    
neighborhoods_venues_sorted.head()


,Neighborhood,1stMost Common Venue,2ndMost Common Venue,3rdMost Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Restaurant,Pharmacy,Beer Bar,Farmers Market,Seafood Restaurant,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Rental Car Location
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Middle Eastern Restaurant,Indian Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Coffee Shop,Nightclub,Athletics & Sports,Restaurant,Italian Restaurant



## Clustering
    

#### Running k-means and clustering it in 5 clusters

In [72]:
from sklearn.cluster import KMeans

kclusters=5

toronto_clustering= toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

#### df which contains top 10 venues and each neighborhood and cluster

In [73]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)



In [74]:
toronto_merged = result_df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on = 'Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1stMost Common Venue,2ndMost Common Venue,3rdMost Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Pub,Park,Breakfast Spot,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1,Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Ramen Restaurant,Hotel
15,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,1,Coffee Shop,Café,Gastropub,Cosmetics Shop,Cocktail Bar,Gym,Farmers Market,Moroccan Restaurant,Department Store,Lingerie Store
19,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
20,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Restaurant,Pharmacy,Beer Bar,Farmers Market,Seafood Restaurant,Jazz Club


## Create Map

In [75]:
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [77]:
latitude = 43.6532
longitude = -79.3832

map_clusters = folium.Map(location = [latitude,longitude],zoom_start = 11)

#set color scheme

x=np.arange(kclusters)

ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0,1,len(ys)))

rainbow=[colors.rgb2hex(i) for i in colors_array]

# add markers to map

markers_colors=[]

for lat, lon, poi,cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    
    folium.CircleMarker([lat,lon],radius =5,popup = label, color = rainbow[cluster-1],fill =True, fill_color = rainbow[cluster-1],fill_opacity=0.75).add_to(map_clusters)

In [79]:
map_clusters